<a href="https://colab.research.google.com/github/TalissaMoura/AmazonC4DatasetAugmented/blob/main/notebooks/2_pipeline_for_calculate_the_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 - Pipeline for measure the metrics for the hard queries generated

In [1]:
!pip install outlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.1 MB/s eta 0:00:00


In [2]:
import os, json, re, torch
import outlines
from huggingface_hub import login,snapshot_download, upload_file, HfApi
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
from pydantic import BaseModel,ValidationError
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
from google.colab import userdata
from datasets import load_dataset
import pathlib
hf_token = userdata.get('HF_TOKEN')
df_amazonc4 = load_dataset('McAuley-Lab/Amazon-C4')['test'].data.table
snapshot_download(
    repo_id='Talissa/AmazonC4Augmented',
    repo_type='dataset',
    local_dir='datasets',
    allow_patterns=['datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet'],
    token=hf_token,
)
# setup hugging faces
hf_api = HfApi(token=hf_token)

README.md: 0.00B [00:00, ?B/s]

test.csv:   0%|          | 0.00/12.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/21223 [00:00<?, ? examples/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

datasets/procesed/hard_negative_queries/(…):   0%|          | 0.00/20.3k [00:00<?, ?B/s]

# 2 - Functions

In [60]:
path_procesed_hnq = pathlib.Path('/content/datasets/contraint_lists_for_generated_queries')

if not pathlib.Path(path_procesed_hnq).exists():
  pathlib.Path.mkdir(path_procesed_hnq,exist_ok=True,parents=True)

class Constraints(BaseModel):
    constraints: list[str]
class Intents(BaseModel):
    intents: list[str]


def extract_json_from_markdown(response_text: str) -> str:
    """Extract JSON from markdown code blocks"""
    # Look for ```json ... ``` or ``` ... ```
    pattern = r'```(?:json)?\s*(.*?)\s*```'
    matches = re.findall(pattern, response_text, re.DOTALL)

    if matches:
        # Return the first JSON block found
        return matches[0].strip()
    else:
        # If no markdown blocks, return original text
        return response_text

def clean_and_parse_json(response_text: str,pydantic_model:BaseModel) -> BaseModel | None:
    # Step 1: Extract JSON from markdown if present
    json_content = extract_json_from_markdown(response_text)

    # Step 2: Remove any remaining backticks or markdown artifacts
    json_content = json_content.strip()

    # Step 3: Handle potential escaping issues
    if json_content.startswith('"') and json_content.endswith('"'):
        # Unescape if it's still a string representation
        try:
            json_content = json.loads(json_content)
        except json.JSONDecodeError:
            pass  # Keep as is if it's not double-escaped

    # Step 4: Parse and validate
    try:
        if isinstance(json_content, str):
            return pydantic_model.model_validate_json(json_content)
        else:
            return pydantic_model.model_validate(json_content)
    except ValidationError as e:
        print(f"Validation error: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return None

from pydantic import ValidationError
import time

def generate_and_parse_with_retry(
    model,
    tokenizer,
    messages,
    pydantic_model,
    max_attempts=3,
    delay=0.5  # optional: to avoid overwhelming GPU or API rate limits
):
    for attempt in range(1, max_attempts + 1):
        # Tokenize and generate
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=2048
        )
        # Trim input part
        generated_ids = [
            output_ids[len(input_ids):]
            for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        print(response)

        # Try parsing
        try:
            parsed = clean_and_parse_json(response, pydantic_model)
            print(parsed)
            if parsed is not None:
                return parsed  # success!
        except ValidationError:
            pass  # will retry

        # Optional: small delay before next attempt
        if attempt < max_attempts:
            time.sleep(delay)

    # All attempts failed
    return None  # or raise an exception, depending on desired behavior


def generate_constraint_list(df_queries,
                             model,
                             tokenizer,
                             pydantic_model=BaseModel,
                             system_template_path=None,
                             user_template_path=None,
                             save_local=True):
  user_template = outlines.Template.from_file(f"{user_template_path}")
  system_template = outlines.Template.from_file(f"{system_template_path}")
  batches = df_queries.to_batches()
  for i, batch in enumerate(batches):
      new_rows = []
      for row in batch.to_pylist():
          query = "-".join(row['hard_negative_query'])

          messages = [
              {"role": "system", "content": system_template()},
              {"role": "user", "content": user_template(user_query=query)}
          ]

          # Try up to 3 times to get a valid parse
          response_treated = generate_and_parse_with_retry(
              model=model,
              tokenizer=tokenizer,
              messages=messages,
              pydantic_model=pydantic_model,
              max_attempts=3
          )

          if response_treated:
            new_rows.append(response_treated)
          else:
            new_rows.append([""])


  batch_pylist = batch.to_pylist()
  new_data = []
  for new_row,data in zip(new_rows,batch_pylist):
      data["constraint_list"] = new_row.__dict__["constraints"]
      new_data.append(data)

  if save_local:
    batch = pa.Table.from_pylist(new_data)
    pq.write_table(batch, f"/content/datasets/contraint_lists_for_generated_queries/all_Beauty_part_{i}.parquet")
    print(f"Batch {i} done.")

def check_constraint_in_lists(row_review,row_contraints):
      list_negative_in_reviews = row_review["negative_points"]
      list_contraints = row_contraints["constraint_list"]
      if list_negative_in_reviews in list_contraints:
        return 1
      else:
        return 0

def load_model(model_name):
  tokenizer = AutoTokenizer.from_pretrained(model_name,fix_mistral_regex=True,device_map="cpu",)
  model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16,device_map="auto")

  # 🔍 Validate placement
  if not next(model.parameters()).is_cuda:
      raise RuntimeError("Model failed to load on GPU!")
  print("✅ Model successfully on GPU.")
  return model,tokenizer


1 - Calculate similarity scores

In [83]:
model_eval = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")

def add_queries_similarity_score(query,hard_negative_query):
  if not query or not hard_negative_query:
    return 0.0
  query_emb = model_eval.encode(query, convert_to_tensor=False)
  hard_negative_query_emb = model_eval.encode(hard_negative_query, convert_to_tensor=False)

  return cosine_similarity([query_emb], [hard_negative_query_emb])[0][0]


2 - Calculate the Constraint Preservation Rate

In [5]:
df_reviews = pq.read_table('/content/datasets/datasets/procesed/hard_negative_queries/all_Beauty_part_0.parquet')

In [88]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_query_restritions_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """

    ```json
      {
        "task": "You are a constraint extractor. Given a product search query, identify and list *only* the explicit or strongly implied characteristics that the user wants to *exclude* or *avoid* in the product. Do not infer preferences unless clearly stated (e.g., 'no plastic', 'avoid heavy items'). Return an empty array if no such constraints exist.",
        "input_format": {
          "query": "string — the user's natural-language product search query"
        },
        "output_type": "json",
        "output_format": {
          "constraints": [
            "string — a phrase that summarize what the user don't want in the product (i.e this product should not be small, this product should avoid the brand A, the user don't want overprice products .. )",
            "Avoid vague terms; prefer concrete, actionable constraints derived directly from the query."
          ]
        },
        "requirements": [
          "Constraints must be phrased as *what to avoid*, not what is desired.",
          "Normalize phrasing (e.g., use 'corded' instead of 'with a cord' if excluding cordless devices).",
          "Do NOT include positive requirements (e.g., 'durable' → ignore unless phrased negatively like 'not fragile').",
          "If ambiguity exists, prefer under-inclusion (i.e., only include unambiguous exclusions)."
        ]
      }
    ```

    """

    with open('/content/prompts/system_prompt_generate_query_restritions_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for constraint preservation rate"
          )

In [38]:
try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"prompts/user_prompt_generate_query_restritions_list"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    user_template = \
    """
    ```json
        {

          "query": {{user_query}}}

        }
    ```
    """

    with open('/content/prompts/user_prompt_generate_query_restritions_list.txt', 'w') as f:
          f.write(user_template)
          upload_file(
              path_or_fileobj=f"/content/prompts/user_prompt_generate_query_restritions_list.txt",
              path_in_repo=f"prompts/user_prompt_generate_query_restritions_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add user prompt for constraint preservation rate"
          )


No files have been modified since last commit. Skipping to prevent empty commit.


In [89]:
print('-- generate constraint list --')
MODEL_NAME = 'Qwen/Qwen2.5-3B-Instruct'
if "model" not in globals():
  model, tokenizer = load_model(MODEL_NAME)
else:
  print("Model already loaded")
generate_constraint_list(df_reviews,
                         model,tokenizer,
                         pydantic_model=Constraints,
                         system_template_path='/content/prompts/system_prompt_generate_query_restritions_list.txt',
                         user_template_path='/content/prompts/user_prompt_generate_query_restritions_list.txt',
                         )

-- generate constraint list --
Model already loaded
```json
{
  "constraints": [
    "too tingling",
    "too small"
  ]
}
```
constraints=['too tingling', 'too small']
```json
{
  "constraints": [
    "doesn't require actual brow hairs for adhesion"
  ]
}
```
constraints=["doesn't require actual brow hairs for adhesion"]
```json
{
  "constraints": [
    "avoid pokey or prickly"
  ]
}
```
constraints=['avoid pokey or prickly']
```json
{
  "constraints": [
    "strong smell",
    "chunky texture",
    "over-exfoliated",
    "scrubbing sensation",
    "rubbing alcohol smell"
  ]
}
```
constraints=['strong smell', 'chunky texture', 'over-exfoliated', 'scrubbing sensation', 'rubbing alcohol smell']
```json
{
  "constraints": [
    "pieces of wood",
    "splinter risks",
    "plain container"
  ]
}
```
constraints=['pieces of wood', 'splinter risks', 'plain container']
```json
{
  "constraints": [
    "only tiny bottles",
    "only 3 small bottles"
  ]
}
```
constraints=['only tiny bottles'

In [ ]:
# adjust query to return the answer as json
# resave the files from the prompts for the first notebook

In [94]:
df_reviews_metrics = pq.read_table('/content/datasets/contraint_lists_for_generated_queries/all_Beauty_part_0.parquet')
print(df_reviews_metrics)

def compute_consistent_score(negative_points, constraint_list):
    # Handle edge cases
    if not negative_points or not constraint_list:
        return 0.0  # or np.nan — up to you

    # Encode all sentences
    neg_embs = model_eval.encode(negative_points, convert_to_tensor=False)
    const_embs = model_eval.encode(constraint_list, convert_to_tensor=False)

    # Compute cosine similarity matrix: shape = (len(neg), len(const))
    sims = cosine_similarity(const_embs,neg_embs)

    # For each constraint, get max similarity over negative embeddings
    max_sims = sims.max(axis=1)  # shape: (len(constraints),)

    # Return the biggest value over the similarities
    return float(np.max(max_sims))

new_rows = []
for batch in df_reviews_metrics.to_batches():
  for row in batch.to_pylist():
    negative_points = row['negative_points']
    constraint_list = row['constraint_list']
    row["consistent_score"] = compute_consistent_score(negative_points, constraint_list)
    # row['consistent_hit'] = 0
    # for negative_point in negative_points:
    #   if negative_point in constraint_list:
    #     row['consistent_score'] += 1
    new_rows.append(row)
# consistent_scores_normalized = [1 if row['consistent_score'] > 1 else 0 for row in new_rows]
batch = pa.Table.from_pylist(new_rows)
# batch = batch.append_column("consistent_score_normalized",pa.array(consistent_scores_normalized))
print(batch)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<element: string>
  child 0, element: string
ori_positive_review_distinct: list<element: string>
  child 0, element: string
positive_points: list<element: string>
  child 0, element: string
negative_points: list<element: string>
  child 0, element: string
hard_negative_query: list<element: string>
  child 0, element: string
constraint_list: list<element: string>
  child 0, element: string
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
ori_semi_positive_review_list: [[["The masks are really good for hydrating skin and moisturizing. They are super easy to use and fit really well. My mother uses this every morning and she highly recommends it.","While the mask was a tiny bit small compared to most brands I have tried, it had plenty of serum and left my skin looking and feeling very hydrated and soothed. I knew it has tea tree oil in it, w

In [92]:
batch.group_by("item_id").aggregate([("consistent_score", "mean")])

pyarrow.Table
item_id: string
consistent_score_mean: double
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
consistent_score_mean: [[0.4633733332157135,0.8018824458122253,0.3065916895866394,0.5009890198707581,0.2630251348018646,0.9260580539703369,0.7434284090995789,0.4418942928314209,0.29536333680152893]]

In [73]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load model ONCE (outside loops!)
# model = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")  # Fast, good for this task

# def compute_consistent_score(negative_points, constraint_list):
#     # Handle edge cases
#     if not negative_points or not constraint_list:
#         return 0.0  # or np.nan — up to you

#     # Encode all sentences
#     neg_embs = model.encode(negative_points, convert_to_tensor=False)
#     const_embs = model.encode(constraint_list, convert_to_tensor=False)

#     # Compute cosine similarity matrix: shape = (len(neg), len(const))
#     sims = cosine_similarity(neg_embs, const_embs)

#     # For each negative point, get max similarity over constraints
#     max_sims = sims.max(axis=1)  # shape: (len(neg),)

#     # Return average of max similarities
#     return float(np.mean(max_sims))

eval_model = SentenceTransformer('all-MiniLM-L6-v2',device="cuda")

new_rows = []
for batch in df_reviews_metrics.to_batches():
  for row in batch.to_pylist():
    negative_points = row['negative_points']
    constraint_list = row['constraint_list']
    row["consistent_score"] =
    # row['consistent_hit'] = 0
    # for constraint in constraint_list:
    #   if constraint in negative_points:
    #     row['consistent_hit'] += 1
    # new_rows.append(row)
# consistent_scores = [1.0 if row['consistent_hit'] >= 1.0 else 0.0 for row in new_rows]
t_batch = pa.Table.from_pylist(new_rows)
# t_batch = t_batch.append_column("consistent_score",pa.array(consistent_scores))
print(t_batch)


# Usage:
# df_reviews_metrics = add_consistent_score(df_reviews_metrics)

pyarrow.Table
item_id: string
ori_semi_positive_review_list: list<item: string>
  child 0, item: string
ori_positive_review_distinct: list<item: string>
  child 0, item: string
positive_points: list<item: string>
  child 0, item: string
negative_points: list<item: string>
  child 0, item: string
hard_negative_query: list<item: string>
  child 0, item: string
constraint_list: list<item: string>
  child 0, item: string
consistent_hit: int64
consistent_score: double
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
ori_semi_positive_review_list: [[["The masks are really good for hydrating skin and moisturizing. They are super easy to use and fit really well. My mother uses this every morning and she highly recommends it.","While the mask was a tiny bit small compared to most brands I have tried, it had plenty of serum and left my skin looking and feeling very hydrated and soothed. I knew it has tea tree o

In [74]:
df_reviews_metrics.group_by("item_id").aggregate([("hard_negative_query", "count")])

pyarrow.Table
item_id: string
hard_negative_query_count: int64
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
hard_negative_query_count: [[1,1,1,1,1,1,1,1,1]]

In [69]:
t_batch.group_by("item_id").aggregate([("consistent_score", "mean")])

pyarrow.Table
item_id: string
consistent_score_mean: double
----
item_id: [["B0B186FXGM","B099KSC4L2","B08MF3W9DZ","B0B34GRFL2","B0B86TKMP8","B0BXXD1FZ4","B08L5KN7X4","B00K7J4S6G","B09TT8ZVPN"]]
consistent_score_mean: [[0.4633733332157135,0,0.2809489071369171,0.5029644966125488,0.2630251348018646,0.7017921209335327,0.6043304800987244,0.3662206530570984,0.32680821418762207]]

3 - Check the user intent score

In [ ]:
import outlines, pathlib

if not pathlib.Path('/content/prompts/').exists():
  pathlib.Path.mkdir('/content/prompts')

try:
  repo_id = "Talissa/AmazonC4Augmented"
  filename = f"/content/prompts/system_prompt_generate_query_restritions_list.txt"
  if not hf_api.file_exists(repo_id=repo_id,filename=filename,repo_type="dataset"):
    raise FileNotFoundError
except FileNotFoundError:
    system_template = \
    """

    ```json
      {
        "task": "You are a constraint extractor. Given a product search query, identify and list *only* the explicit or strongly implied characteristics that the user wants to *exclude* or *avoid* in the product. Do not infer preferences unless clearly stated (e.g., 'no plastic', 'avoid heavy items'). Return an empty array if no such constraints exist.",
        "input_format": {
          "query": "string — the user's natural-language product search query"
        },
        "output_type": "json",
        "output_format": {
          "constraints": [
            "string — a specific, concise product attribute to exclude (e.g., 'plastic material', 'wireless', 'under 500g', 'battery-powered')",
            "Avoid vague terms; prefer concrete, actionable constraints derived directly from the query."
          ]
        },
        "requirements": [
          "Constraints must be phrased as *what to avoid*, not what is desired.",
          "Use neutral, descriptive language (e.g., 'contains BPA' not 'unsafe chemicals').",
          "Normalize phrasing (e.g., use 'corded' instead of 'with a cord' if excluding cordless devices).",
          "Do NOT include positive requirements (e.g., 'durable' → ignore unless phrased negatively like 'not fragile').",
          "If ambiguity exists, prefer under-inclusion (i.e., only include unambiguous exclusions)."
        ]
      },
      "examples": [
        {
          "input": { "query": "I need a laptop stand, but no plastic parts and preferably not too heavy" },
          "output": { "constraints": ["plastic parts", "heavy"] }
        },
        {
          "input": { "query": "Looking for a quiet mechanical keyboard without RGB lighting" },
          "output": { "constraints": ["RGB lighting", "loud"] }
        },
        {
          "input": { "query": "A budget-friendly tablet for reading" },
          "output": { "constraints": [] }
        }
      ]
    ```

    """

    with open('/content/prompts/system_prompt_generate_user_intent_list.txt', 'w+') as f:
          f.write(system_template)
          f.close()
          upload_file(
              path_or_fileobj=f"/content/prompts/system_prompt_generate_user_intent_list.txt",
              path_in_repo=f"prompts/system_prompt_generate_user_intent_list.txt",
              repo_id="Talissa/AmazonC4Augmented",
              repo_type="dataset",
              commit_message="Add system prompt for user intent score rate"
          )

In [81]:
# Force garbage collection to free CPU memory
import gc
gc.collect()

# Clear GPU cache to free up CUDA memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print("✅ GPU cache cleared")
else:
    print("CUDA not available")

print("✅ Cleanup complete - model/tokenizer removed from CPU/GPU")

✅ GPU cache cleared
✅ Cleanup complete - model/tokenizer removed from CPU/GPU
